# Exploratory Data Analysis <a class="anchor"><a id='PVA'></a></b><br>
Authors:<br><br>
Student Name - Gonçalo Custódio<br>
- Student id - 20211643<br>
- Contact e-mail - 20211643@novaims.unl.pt<br>
  
Student Name - Diogo Correia<br>
- Student id - 20211586<br>
- Contact e-mail - 20211586@novaims.unl.pt<br>
  
Student Name - João Santos<br>
- Student id - 20211691<br>
- Contact e-mail - 20211691@novaims.unl.pt<br>
  
Student Name - Nuno Bernardino<br>
- Student id - 20211546<br>
- Contact e-mail - 20211546@novaims.unl.pt<br>

In [ ]:
#!pip install plotly

In [ ]:
#import sqlite3
#import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import math
from scipy.stats import chi2_contingency

In [ ]:
data = pd.read_excel("DM2425_ABCDEats_DATASET.xlsx", sheet_name="DM2425_ABCDEats_DATASET")

# 1. Deep Exploration

**Initial Analysis**

To kick off our deep exploration, we’ll use the `data.info()` command to get an overview of the dataset. This command provides essential information, including the number of entries, column names, non-null counts, and data types for each variable. This quick summary will allow us to identify any missing values, spot potential data type issues, and gain a high-level understanding of the dataset's structure, setting the stage for further analysis.



In [ ]:
data.info()

**To visualize and get a sense of the data**

To begin familiarizing ourselves with the dataset, we’ll use the `data.head()` command, which displays the first few rows of the data. This preview allows us to quickly see the contents, spot-check variable values, and gain an initial sense of the data's structure and content. By viewing these rows, we can start to assess the data format, variable types, and any immediate patterns or issues that stand out.


In [ ]:
data.head()

**Check data types of our variables**

In this step, we’ll use the `data.dtypes` command to examine the data types of each variable in our dataset. This overview will confirm if the variables are appropriately typed (e.g., integers, floats, objects) and will help us spot any inconsistencies or unexpected types that might require adjustment. Understanding the data types at this stage is crucial, as it guides us in selecting suitable preprocessing and analysis techniques for each variable.


In [ ]:
data.dtypes

Since we don't agree with the fact that the variables in question (last_promo, payment_method and customer_region) are of type object we will change them to category in order to facilitate future analysis:

In [ ]:
data['last_promo'] = data['last_promo'].astype('category')
data['payment_method'] = data['payment_method'].astype('category')
data['customer_region'] = data['customer_region'].astype('category')

We will divide the variables into lists for categorical and numerical variables to facilitate future interactions:

In [ ]:
category_var = ['customer_region', 'last_promo', 'payment_method']

In [ ]:
number_var = ['customer_age', 'vendor_count', 'product_count', 'is_chain', 'first_order', 'last_order', 
              'CUI_American', 'CUI_Asian', 'CUI_Beverages', 'CUI_Cafe', 'CUI_Chicken Dishes', 'CUI_Chinese', 
              'CUI_Desserts', 'CUI_Healthy', 'CUI_Indian', 'CUI_Italian', 'CUI_Japanese', 'CUI_Noodle Dishes', 
              'CUI_OTHER', 'CUI_Street Food / Snacks', 'CUI_Thai', 'DOW_0', 'DOW_1', 'DOW_2', 'DOW_3', 'DOW_4', 
              'DOW_5', 'DOW_6', 'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9', 
              'HR_10', 'HR_11', 'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 
              'HR_21', 'HR_22', 'HR_23']

# 2. Summary of Important Statistics

In this section, we’ll start by generating a statistical summary using the `data.describe()` command. This summary provides key descriptive statistics for each numerical variable, such as the mean, standard deviation, minimum, and maximum values. By reviewing these statistics, we can understand the central tendencies, dispersion, and overall range of our data. 

Following this, we’ll delve into more detailed analyses, including frequency distributions for categorical variables, box plots, and histograms, each accompanied by summary boxes with key metrics. Together, these statistics and visualizations give us a comprehensive view of the data, setting a foundation for more advanced analyses.



In [ ]:
data.describe()

Sendo assim podemos verificar que algumas variáveis tem outliers

Para isso iremos fazer uma função para identificar quantos outliers temos:

In [ ]:
def detect_outliers_summary(df, columns):
    outliers_data = []

    numeric_columns = df[columns].select_dtypes(include=['number']).columns
    
    for col in numeric_columns:
        Q1 = df[col].quantile(0.25) 
        Q3 = df[col].quantile(0.75)  
        IQR = Q3 - Q1 

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        
        outliers_data.append({'Coluna': col, 'Número de Outliers': outlier_count})
    
    outliers_summary = pd.DataFrame(outliers_data)
    return outliers_summary

outliers_summary_table = detect_outliers_summary(data, number_var)
print(outliers_summary_table)

Como podemos verificar existem bastantes outliers, coisa que mais para a frente iremos tratar.

For numerical variables, the statistics above are very effective, but for categorical variables we must count the frequency of each value

**Frequency of each value in Categorical Variables**

To gain insights into the distribution of categorical variables, we will compute the frequency of each unique value in these columns. By understanding the frequency of categories, we can identify dominant categories, assess data balance, and detect potential issues such as sparse categories or outliers that may affect our analysis.


In [ ]:
for column in category_var:
    frequency_percentage = data[column].value_counts(normalize=True) * 100
    result = pd.DataFrame({'Value': frequency_percentage.index, 'Percentage': frequency_percentage.values})

    print(f"\033[1mColumn '{column}'\033[0m:")
    print(result)
    print("\n") 

**Now for a better visualization of the data and their respective statistics, let's plot some graphs:**

**2.1 Box-Plot**

Box plots allow us to visualize the spread, central tendency, and range of values for each variable. For each variable, we will generate a box plot that includes a summary box indicating the average value, median, minimum, and maximum values. This helps us identify any skewness, outliers, or unusual patterns in the data, which are essential for informed data preprocessing and decision-making in our data mining process.


In [ ]:
sns.set_style("white")

numeric_columns = [col for col in data.select_dtypes(include=['int64', 'float64']).columns if data[col].notna().any()]

n_cols = 3

n_rows = math.ceil(len(numeric_columns) / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 6))

axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.boxplot(data=data, y=col, ax=axes[i])
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Value")

    mean_value = data[col].mean()
    median_value = data[col].median()
    min_value = data[col].min()
    max_value = data[col].max()

    stats_text = (f'Average: {mean_value:.2f}\n'
                  f'Median: {median_value:.2f}\n'
                  f'Min: {min_value:.2f}\n'
                  f'Max: {max_value:.2f}')
    
    axes[i].text(1.05, 0.5, stats_text, transform=axes[i].transAxes, 
                  fontsize=12, verticalalignment='center', bbox=dict(boxstyle='round', alpha=0.1))

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

**2.2 Histrogram**

Histograms provide a visual representation of the distribution of values for each variable. For each variable, we will create a histogram along with a summary box that displays the average value, median, minimum, and maximum values. These histograms help us understand the frequency distribution and shape (e.g., normal, skewed) of each variable, offering insights that will guide us in selecting appropriate analytical methods.


In [ ]:
sns.set_style("white")

numeric_columns = [col for col in data.select_dtypes(include=['int64', 'float64']).columns if data[col].notna().any()]

n_cols = 3

n_rows = math.ceil(len(numeric_columns) / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 6))

axes = axes.flatten()

for i, col in enumerate(numeric_columns):
    sns.histplot(data[col], bins=30, ax=axes[i], stat='count') 
    axes[i].set_title(col)
    axes[i].set_xlabel("Value")
    axes[i].set_ylabel("Frequency")

    mean_value = data[col].mean()
    median_value = data[col].median()
    min_value = data[col].min()
    max_value = data[col].max()

    stats_text = (f'Average: {mean_value:.2f}\n'
                  f'Median: {median_value:.2f}\n'
                  f'Min: {min_value:.2f}\n'
                  f'Max: {max_value:.2f}')
    
    axes[i].text(1.05, 0.5, stats_text, transform=axes[i].transAxes, 
                  fontsize=12, verticalalignment='center', bbox=dict(boxstyle='round', alpha=0.1))

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# 3. Identify Trends, Patterns, or Anomalies
In this chapter, our objective is to uncover underlying trends, patterns, and anomalies within the data. By systematically analyzing these elements, we can detect recurring behaviors, identify correlations, and highlight unusual values or outliers that might impact our analysis. Understanding these factors is essential in a data mining project, as they often reveal deeper insights and influence the accuracy of predictive models. This analysis will inform our feature engineering choices and guide us toward meaningful data transformations, ultimately enhancing the value and interpretability of our results.


### 3.1 Anomalies

In this section, we focus on detecting anomalies within our dataset. Anomalies, or outliers, are values that deviate significantly from the majority of observations and may indicate data entry errors, rare events, or unique patterns. Identifying these values early on is crucial, as they can impact model performance and lead to misleading insights if not handled properly.


When we open the dataset, we notice that there are missing values and hyphen values, because there are blank values and others with only “-”. We need to correct this:

In [ ]:
missing_values = data.isnull().sum()
hyphen_counts = (data == '-').sum()

total_rows = data.shape[0]
missing_percentage = (missing_values / total_rows) * 100
hyphen_percentage = (hyphen_counts / total_rows) * 100

missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Hyphen Values': hyphen_counts,
    'Missing %': missing_percentage,
    'Hyphen %': hyphen_percentage
})

missing_summary

In [ ]:
# Filtrar as linhas onde first_order é ausente e last_order é diferente de 0
missing_first_order = data[data['first_order'].isnull() & (data['last_order'] != 0)]

# Verificar se existe alguma linha com essa condição e exibir o resultado
if not missing_first_order.empty:
    print("Linhas onde 'first_order' é missing e 'last_order' é diferente de 0:")
    print(missing_first_order)
else:
    print("Não existem linhas onde 'first_order' é missing e 'last_order' é diferente de 0.")

In [ ]:
data["HR_0"].unique()

Futuramente iremos tratar dos missing values e dos hyphen values

*Relativamente à coluna customer_region, entendemos que os hyphen values presentes podem representar valores desconhecidos, por isso, iremos trocar os hyphen values para "Missing"*

*Relativamente à coluna customer_age, entendemos que os missing values presentes representam que a idade não foi informada pelo cliente, por isso, iremos utilizar uma decision tree para dar input a valores ausentes*

*Relativamente à coluna first_order, entendemos que os missing values presentes representam 0 pois sempre que isso acontece, o last_order também é 0*

*Relativamente à coluna last_promo, verificamos que existe mais de 50% dos valores com hyphen, o que nos deixa a pensar se dropamos a coluna ou não. Caso não o façamos, iremos trocar esses hyphen values por "No promo"*

*Relativamente à coluna HR_0, entendemos que como existem 0 values e NaN values, os NaN significam falta de informação, e como os 0 values não são relevantes para a nossa análise, dropamos a coluna em questão*

In [ ]:
data.drop(columns=['HR_0'], inplace=True)

if 'HR_0' in number_var:
    number_var.remove('HR_0')

We need to verify if there are several lines with the same customer_id

In [ ]:
first_column = data.columns[0] 
duplicates = data[data.duplicated(subset=[first_column], keep=False)] 

print(f"Repeats values in '{first_column}' column:")
print(duplicates[[first_column]].drop_duplicates()) 
print(f"\nTotal duplicates in '{first_column}' are {duplicates.shape[0]}")

We will check whether the repeated rows with the same id have exactly the same values in the other variables or not

In [ ]:
target_id = ["742ca068fc","b55012ee1c","24251eb7da","671bf0c738","df91183978","6bbf5f74cd","8aa9bbc147","cf563a0a98","201a13a34d","06018a56be","fac7984c0d","b8e7a643a4","cc08ef25ce"]

matching_rows = data[data[data.columns[0]].isin(target_id)]

pd.set_option("display.max_columns", None)  
pd.set_option("display.width", None) 

matching_rows

We can see that the 26 lines with the repeated id are exactly the same in all the variables. So let's get rid of these repeated lines and keep just one:

In [ ]:
data = data.drop_duplicates(subset=[first_column], keep='first')

data

### 3.2 Correlation

In this section, we examine the correlations between variables to identify relationships and dependencies within the dataset. By analyzing these correlations, we can detect variables that move together, uncover potential redundancies, and gain insights into which variables may have the most predictive power. This step is essential for refining our feature selection and improving model accuracy.


**Correlation table between Numeric Variables**

In [ ]:
correlation_matrix = data[number_var].corr()

high_corr_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.5 or abs(correlation_matrix.iloc[i, j]) < -0.5: 
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Variable 1', 'Variable 2', 'Correlation'])

high_corr_df

**Heatmaps**

Since there are quite a few numerical variables, we'll divide them into groups so that the heatmap isn't overloaded with unreadable information

**Heatmap of Demographic variables**

In [ ]:
demographic_vars = ['customer_age', 'vendor_count', 'product_count', 'is_chain']

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix[demographic_vars].loc[demographic_vars], annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.show()

**Heatmap of CUI variables**

In [ ]:
#cui_vars = [
  #  'CUI_American', 'CUI_Asian', 'CUI_Beverages', 'CUI_Cafe', 'CUI_Chicken Dishes', 
  #  'CUI_Chinese', 'CUI_Desserts', 'CUI_Healthy', 'CUI_Indian', 'CUI_Italian', 
 #   'CUI_Japanese', 'CUI_Noodle Dishes', 'CUI_OTHER', 'CUI_Street Food / Snacks', 'CUI_Thai'
#]

#plt.figure(figsize=(10, 8))
#sns.heatmap(correlation_matrix[cui_vars].loc[cui_vars], annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
#plt.show()

**Heatmap of DOW variables**

In [ ]:
#dow_vars = ['DOW_0', 'DOW_1', 'DOW_2', 'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6']

#plt.figure(figsize=(10, 8))
#sns.heatmap(correlation_matrix[dow_vars].loc[dow_vars], annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
#plt.show()


**Heatmap of HR variables**

In [ ]:
hr_vars = [
    'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 
    'HR_6', 'HR_7', 'HR_8', 'HR_9', 'HR_10', 'HR_11', 
    'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 
    'HR_18', 'HR_19', 'HR_20', 'HR_21', 'HR_22', 'HR_23'
]

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix[hr_vars].loc[hr_vars], annot=True, fmt=".2f", cmap='coolwarm', 
            mask=correlation_matrix[hr_vars].loc[hr_vars] < 0.1, square=True, cbar_kws={"shrink": .8})
plt.show()


**Heatmap of CUI Foods and the Days of the Week**

In [ ]:
food_columns = [
    'CUI_American', 'CUI_Asian', 'CUI_Beverages', 'CUI_Cafe', 'CUI_Chicken Dishes', 
    'CUI_Chinese', 'CUI_Desserts', 'CUI_Healthy', 'CUI_Indian', 'CUI_Italian', 
    'CUI_Japanese', 'CUI_Noodle Dishes', 'CUI_OTHER', 'CUI_Street Food / Snacks', 'CUI_Thai'
]

day_columns = ['DOW_0', 'DOW_1', 'DOW_2', 'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6']

selected_columns = food_columns + day_columns
correlation_matrix = data[selected_columns].corr()

plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
            mask=correlation_matrix < 0.1, square=True, cbar_kws={"shrink": .8})
plt.show()

**Correlation between categorical variables**

**Chi-Squared**

In [ ]:
def chi2_test(var1, var2):
    contingency_table = pd.crosstab(data[var1], data[var2])
    chi2, p, _, _ = chi2_contingency(contingency_table)
    return chi2, p

In [ ]:
combinations = [
    ("customer_region", "last_promo"),
    ("customer_region", "payment_method"),
    ("last_promo", "payment_method")
]

results = []
for var1, var2 in combinations:
    chi2_value, p_value = chi2_test(var1, var2)
    results.append({"Variable 1": var1, "Variable 2": var2, "Chi-Squared": chi2_value, "p-value": p_value})

resultss = pd.DataFrame(results)
resultss

*It is important to note that the p-value should be low, as there is sufficient evidence to reject the null hypothesis!*

# 4. Create New Features

In this chapter, we focus on creating new features to enhance the dataset’s predictive power and capture more meaningful patterns. By engineering features based on existing variables, we can reveal hidden relationships, improve model performance, and better address our project objectives. This process is a key step in preparing our data for analysis, enabling us to extract deeper insights from the data.


We decided to make for now 5 new features:

Age Category: This segmentation can be valuable in tailoring promotions and communications for specific age demographics.

In [ ]:
# Age Category
def age_group(age):
    if age < 18:
        return "Young"
    elif 18 < age < 65:
        return "Adult"
    else:
        return "Senior"

data["age_category"] = data["customer_age"].apply(age_group)

Order Frequency per Customer: This feature can help ABCDEats understand the engagement level of each customer.

In [ ]:
# Order Frequency Per Customer
def calculate_order_frequency(df):
    return df.groupby('customer_id')['order_id'].transform('count')

data['order_frequency_per_customer'] = calculate_order_frequency(data)

Average Order Value: This metric allows segmentation of customers based on spending patterns, helping to identify high-value customers.

In [ ]:
# Average Order Value
def calculate_average_order_value(df):
    return df.groupby('customer_id')['product_count'].transform('mean')

data['average_order_value'] = calculate_average_order_value(data)

High-engagement Customer Flag: This binary flag identifies customers whose order frequency or average order value is above the overall average. Customers with a high engagement flag could be potential candidates for loyalty programs or targeted retention strategies.

In [ ]:
# High-Engagement Customer Flag
def high_engagement_flag(df):
    avg_order_frequency = df['order_frequency_per_customer'].mean()
    avg_order_value = df['average_order_value'].mean()
    return ((df['order_frequency_per_customer'] > avg_order_frequency) | 
            (df['average_order_value'] > avg_order_value)).astype(int)

data['high_engagement_customer'] = high_engagement_flag(data)

Não sei o que meter aqui malta

In [ ]:
# Time between first purchase and last purchase
data["days_between_first_last_purchase"] = data["last_order"] - data["first_order"]

In [ ]:
data.head()